In [1]:
from covid.common import SparkSessionService

In [2]:
workspace_path = "COVID-DATABRICKS"
database_path = f"/{workspace_path}/.data"
spark_session_service = SparkSessionService()
spark = spark_session_service.get_spark_session()

:: loading settings :: url = jar:file:/opt/conda/envs/pyspark/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1ec20d1b-7f87-4cd0-a3d9-e1af2c7122a5;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 121ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.4.0 from central in [default]
	io.delta#delta-storage;2.4.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0  

In [3]:
from covid.bronze import ingest_raw_files

landing_path = f"{database_path}/raw/"
bronze_path = f"{database_path}/bronze/covid"

ingest_raw_files(spark, landing_path, bronze_path)

25/11/17 22:32:47 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
covid_df = spark.read.format("delta").load(bronze_path)
covid_df.show()

+------+------+---------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+
|regiao|estado|municipio|coduf|codmun|codRegiaoSaude|nomeRegiaoSaude|      data|semanaEpi|populacaoTCU2019|casosAcumulado|casosNovos|obitosAcumulado|obitosNovos|Recuperadosnovos|emAcompanhamentoNovos|interior/metropolitana|
+------+------+---------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+
|Brasil|  null|     null|   76|  null|          null|           null|2024-07-01|       27|       210147125|   3.8832913E7|         0|         712537|          0|        38110837|                 9539|                  null|
|Brasil|  null|     null|   76|  null|          null|           null|2024-07-02|       27|       2101471

In [5]:
covid_df.count()

11177059

In [6]:
import pyspark.sql.functions as F

covid_df.where(F.col("municipio").isNotNull()).count()

11078730